# Q2a

In [1]:
import numpy as np

def bandit(arm):   #function to simulate the two-armed slot machine, taking the arm we want to play as input
                   #and returns a sample from a Bernoulli distrubution for each arm 
    mu1 = 0.6
    mu2 = 0.4
    
    if arm == 1:   
        return np.random.binomial(1, mu1)
    elif arm == 2:
        return np.random.binomial(1, mu2)

In [2]:
def epsilon_decreasing(n, epsilon_sequence):     

    def choose_arm(epsilon, wins_1, plays_1, wins_2, plays_2):   #function that returns which arm we should play at each step
                                                                 #of the epsilon decreasing strategy, given a probability   
                                                                 #epsilon and the number of plays and wins for each arm
        
        if np.random.binomial(1, epsilon) == 1:   #returns a uniformly random arm with probability epsilon               
            arm = np.random.binomial(1, 0.5) + 1
            return arm
        
        else:                           #else, plays the arm with the best win rate, or a random arm if the win rates are equal
            win_rate_1 = wins_1/plays_1
            win_rate_2 = wins_2/plays_2
            if win_rate_1 > win_rate_2:
                return 1
            elif win_rate_2 > win_rate_1:
                return 2
            else:
                return np.random.binomial(1,0.5) + 1
        
    arms = np.zeros(n, int)            
    rewards = np.zeros(n, int)         
    
    arms[0] = 1                        #play the first arm once
    rewards[0] = bandit(1)

    arms[1] = 2                        #play the second arm once
    rewards[1] = bandit(2)

    plays_1 = 1; plays_2 = 1               #these variables are passed to choose_arm for efficiency
    wins_1 = rewards[0]; wins_2 = rewards[1]

    for i in range(2, n):              #then, follow the epsilon decreasing strategy until n steps have occured
        
        arm = choose_arm(epsilon_sequence[i], wins_1, plays_1, wins_2, plays_2)    
        
        if arm == 1:                  
            arms[i] = 1
            plays_1 += 1
            r = bandit(1)
            rewards[i] = r            #record the arm played and reward gained at each step
            wins_1 += r
        else:
            arms[i] = 2
            plays_2 += 1
            r = bandit(2)
            rewards[i] = r
            wins_2 += r
    
    return (arms, rewards)      



In [3]:
def Thompson_sampling(n):   
    
    alpha = 1
    beta = 1

    arms = np.zeros(n, int)           
    rewards = np.zeros(n, int)

    s1 = 0; f1 = 0; s2 = 0; f2 = 0    #s1 and f1 are successes and failures for arm 1, vice versa for arm 2,  
                                      #as in the lecture slides
    for i in range(n):
        
        m1 = np.random.beta(alpha + s1, beta + f1)   #at each step, sample from a beta distribution
        m2 = np.random.beta(alpha + s2, beta + f2)
        
        if m1 >= m2:             #play arm 1 if m1 >= m2
            arms[i] = 1
            r = bandit(1)        
            rewards[i] = r
            if r == 1: 
                s1 += 1
            else: f1 += 1
                
        else:                   #play arm 2 otherwise
            arms[i] = 2
            r = bandit(2)       
            rewards[i] = r
            if r == 1: 
                s2 += 1
            else: f2 += 1

    return (arms, rewards)      #return the arms played and rewards gained

# Q2b 

Given $\epsilon _{n} = \min(1, C/n) $, the epsilon decreasing algorithm will play completely randomly for $\left \lfloor C \right \rfloor$ steps because if $n$ is less than $C$, then $C/n >1$ and so $\epsilon_n = 1$. Then, the arm with the best win rate will be chosen more often at each step, until it is chosen the vast majority of the time. Thus, for small values of C, we do not play each arm many times before deciding which arm is best, so we may observe by chance that the arm with the worse "true win rate" actually has a better observed win rate, meaning that we may play the worse arm the majority of the time, leading to low average reward over time. This becomes increasingly unlikely as $C$ increases, but then we are also playing randomly for a long time for large values of $C$, leading to a large lost reward early on, i.e a higher regret. So its a trade-off between making sure the arm we choose is actually best and getting the best rewards early on.

We can observe this in our implementation if we run the epsilon decreasing algorithm 20 times with a small $C$ and 20 times with a high $C$ and calculate the average reward for each run

In [12]:
n = 10000
C = 1            #low value of C
seq = []
for i in range(1,n+1):
    seq.append(min(1, C/(i)))
    
for i in range(20):
    print(sum(epsilon_decreasing(n, seq)[1])/n)

0.587
0.5956
0.6007
0.5988
0.6035
0.594
0.6049
0.3966
0.5258
0.5991
0.5994
0.6015
0.6066
0.3955
0.5996
0.6007
0.5948
0.3957
0.4022
0.6054


We want our average reward to be as close as possible to $0.6$ because in this case, $\max(\mu _1, \mu_2) = 0.6$. In most of the above runs, our average reward is close to 0.6, but in a few, it is close to 0.4 which means that the worse arm was played the majority of the time because it had (by chance) a better empirical win rate early on. In other words, when $C$ is small, the algorithm can sometimes "choose" the wrong arm and stick with it.

Now lets see the results for a high value of $C$

In [11]:
n = 10000
C = 100           #high value of C
seq = []
for i in range(1,n+1):
    seq.append(min(1, C/(i)))
    
for i in range(20):
    print(sum(epsilon_decreasing(n, seq)[1])/n)

0.5994
0.5898
0.5974
0.5857
0.5942
0.5896
0.6029
0.5942
0.5925
0.5951
0.6022
0.5911
0.5926
0.5909
0.5904
0.5951
0.5994
0.5883
0.5957
0.5925


We see that the true best arm was played most of the time here in every run, because we had many early steps to collect information about each arm, but the average reward is slightly lower than the runs which correctly identified the best arm in the runs with small $C$. This shows us that the regret is higher with a large value of $C$ because of the longer time spent playing randomly early on.

# Q2c

$\frac{C}{n^2}$ decreases much faster than $\frac{C}{n}$, so we would expect to see a similar pattern to above, but more extreme, with the algorithm "settling" on one arm sooner for small values of $C$. Effectively, this is closer to the "$\epsilon$-first" strategy. Let's repeat the above tests with the same values of $C$ to see this effect.

In [20]:
n = 10000
C = 1            #low value of C
seq = []
for i in range(1,n+1):
    seq.append(min(1, C/(i**2)))
    
for i in range(20):
    print(sum(epsilon_decreasing(n, seq)[1])/n)

0.599
0.5931
0.4008
0.5962
0.5978
0.6003
0.4027
0.6002
0.601
0.4
0.3987
0.602
0.5962
0.6054
0.5983
0.6113
0.5999
0.3911
0.6003
0.4002


Here, it has chosen the wrong arm 6 times out of 20.

In [15]:
n = 10000
C = 100           #high value of C
seq = []
for i in range(1,n+1):
    seq.append(min(1, C/(i**2)))
    
for i in range(20):
    print(sum(epsilon_decreasing(n, seq)[1])/n)

0.5939
0.6006
0.596
0.6023
0.5978
0.5968
0.5855
0.604
0.6061
0.6022
0.5941
0.6034
0.6061
0.6024
0.5911
0.6007
0.5943
0.6026
0.5899
0.3978


Even for a large value of $C$, the algorithm chose the wrong arm once, although when choosing the correct arm, the average rewards appear higher than the first $\epsilon$ sequence because once choosing the right arm, the chances of playing randomly become extremely small very quickly.

# Q2d

Lets run our Thompson sampling algorithm and compute average rewards for comparison.

In [21]:
n = 10000
for i in range(20):
    print(sum(Thompson_sampling(n)[1])/n)

0.6012
0.593
0.6027
0.6044
0.6053
0.6004
0.5975
0.5969
0.6025
0.6028
0.5998
0.6114
0.5991
0.6018
0.5984
0.5942
0.6051
0.6031
0.6054
0.6024


This strategy appears to work very well because all the average rewards are extremely close to $0.6$. Thompson sampling does not exhibit the same "choosing the wrong arm" phenomenon as the $\epsilon$-decreasing strategy. The average rewards also seem to be higher or around the same as $\epsilon$-decreasing, when ignoring the runs where the wrong arm was chosen. To see which algorithm is better, lets run the Thompson algorithm , the first $\epsilon$-decreasing with large $C$, and the second $\epsilon$-decreasing with large $C$ each 100 times and compute an "average of averages" for the rewards.

In [22]:
n = 10000
C == 100
total = 0                              #Thompson sampling
for i in range(100):
    total += sum(Thompson_sampling(n)[1])/n

print(total/100)



seq = []
for i in range(1,n+1):
    seq.append(min(1, C/(i)))          #epsilon decreasing with C/n
total = 0
for i in range(100):
    total += sum(epsilon_decreasing(n, seq)[1])/n

print(total/100)


seq = []
for i in range(1,n+1):
    seq.append(min(1, C/(i**2)))         #epsilon decreasing with C/n^2
total = 0
for i in range(100):
    total += sum(epsilon_decreasing(n, seq)[1])/n

print(total/100)

0.5983670000000003
0.5652259999999999
0.5563050000000002


This shows us that Thompson sampling is best because it has the highest average reward over many runs, which is consistent with the theory. This also demonstrates that sequences that decrease slower are better for the $\epsilon$-decreasing algorithm, because the average reward over many runs is higher for the first sequence than the second.